In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)


X = df.drop('Class', axis=1)
y = df['Class']

smote = SMOTE(random_state=89)
X_res, y_res = smote.fit_resample(X, y)
balanced_df = pd.concat([pd.DataFrame(X_res), pd.Series(y_res, name='Class')], axis=1)

In [2]:
import numpy as np


n = int(np.ceil((1.96**2 * 0.5 * 0.5) / (0.05**2)))
s1 = balanced_df.sample(n=n, random_state=55)



k = len(balanced_df) // n
s2 = balanced_df.iloc[::k][:n]

s3 = balanced_df.groupby('Class', group_keys=False).apply(lambda x: x.sample(n=n//2, random_state=20))

balanced_df['cluster'] = np.repeat(np.arange(20), len(balanced_df)//20 + 1)[:len(balanced_df)]
chosen_clusters = np.random.choice(range(20), size=5, replace=False)
s4 = balanced_df[balanced_df['cluster'].isin(chosen_clusters)].drop('cluster', axis=1)


s5 = balanced_df.sample(n=n, replace=True, random_state=30)

samples = [s1, s2, s3, s4, s5]

/tmp/ipython-input-803968172.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  s3 = balanced_df.groupby('Class', group_keys=False).apply(lambda x: x.sample(n=n//2, random_state=20))


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    "M1": LogisticRegression(C=0.78, solver='liblinear', max_iter=1500), # Adjusted C value
    "M2": RandomForestClassifier(n_estimators=120, max_depth=12, random_state=12), # Changed estimators and depth
    "M3": SVC(kernel='poly', degree=3, probability=True), # Changed kernel to polynomial
    "M4": DecisionTreeClassifier(criterion='entropy', min_samples_split=10), # Changed splitting criteria
    "M5": KNeighborsClassifier(n_neighbors=6, weights='distance') # Changed neighbors and weights
}

results = {}

for model_name, model in models.items():
    model_accuracies = []
    for i, sample in enumerate(samples):
        X_sample = sample.drop('Class', axis=1)
        y_sample = sample['Class']

        X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=50)

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        acc = accuracy_score(y_test, predictions) * 100
        model_accuracies.append(round(acc, 2))

    results[model_name] = model_accuracies

final_table = pd.DataFrame(results, index=['Simple_random', 'systematic', 'stratified', 'cluster', 'bootstramp']).T
print(final_table)

    Simple_random  systematic  stratified  cluster  bootstramp
M1          97.40       93.51       88.31    93.51       97.40
M2          97.40      100.00       98.70    98.70       98.70
M3          66.23       74.03       67.53    87.01       66.23
M4          96.10       98.70       94.81    94.81       97.40
M5          79.22       77.92       81.82    92.21       92.21


In [4]:
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

ml_models = {
    "M1 (ExtraTrees)": ExtraTreesClassifier(n_estimators=150, criterion='entropy', random_state=65),
    "M2 (AdaBoost)": AdaBoostClassifier(n_estimators=100, learning_rate=0.85, random_state=65),
    "M3 (NaiveBayes)": GaussianNB(),
    "M4 (LDA)": LinearDiscriminantAnalysis(),
    "M5 (MLP)": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1500, random_state=65)
}

results_data = {}

for model_name, model in ml_models.items():
    model_accuracies = []

    for i, current_sample in enumerate([s1, s2, s3, s4, s5]):

        X_sample = current_sample.drop('Class', axis=1)
        y_sample = current_sample['Class']

        X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.25, random_state=42)

        model.fit(X_train, y_train)

        predictions = model.predict(X_test)
        acc = accuracy_score(y_test, predictions) * 100
        model_accuracies.append(round(acc, 2))

    results_data[model_name] = model_accuracies

final_comparison_table = pd.DataFrame(results_data,
                                     index=['Sampling1', 'Sampling2', 'Sampling3', 'Sampling4', 'Sampling5']).T

print("\n--- Final Sampling vs Model Accuracy Table ---")
print(final_comparison_table)


--- Final Sampling vs Model Accuracy Table ---
                 Sampling1  Sampling2  Sampling3  Sampling4  Sampling5
M1 (ExtraTrees)     100.00      96.91      97.92     100.00      98.97
M2 (AdaBoost)        95.88      96.91      91.67      98.97      96.91
M3 (NaiveBayes)      84.54      82.47      79.17      81.44      95.88
M4 (LDA)             89.69      85.57      87.50      89.69      96.91
M5 (MLP)             93.81      90.72      89.58      92.78      98.97
